In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
import json
import scipy.stats
from torchvision import datasets, transforms

a=0.5
g=np.pi/2

bias=False
num_shots=15
n_epochs = 100
step=100
batch_size_train = 64
batch_size_test = 10000
learning_rate = 1e-2
momentum = 0.9
input_size=28**2
output_size=10
depth=2
width=10

train_size=5000
test_size=10000

class WMeasure_Rot_Fun(autograd.Function):
    @staticmethod
    def forward(ctx,input,angles,last_res,a,g):
        ctx.save_for_backward(input)
        
        if a==0.0:
            thetas=torch.pi/2*(last_res-torch.sign(input))
        else:
            thetas=torch.pi/2*(last_res-F.tanh(input/a))

        temp_angles=angles+thetas
        
        betas=torch.sin(temp_angles/2)
        exp_z=torch.cos(temp_angles/2)**2-betas**2
        exp_z_anc=exp_z*torch.sin(g)

        probs=torch.stack([(1+exp_z_anc)/2,(1-exp_z_anc)/2],2)
        probs = torch.clamp(probs, min=0)
        outcomes=torch.tensor([1.0,-1.0])
        meas_res=outcomes[torch.distributions.Categorical(probs,validate_args=False).sample()]
        last_res.copy_(meas_res)
        
        side=betas*torch.sqrt((1-meas_res*torch.sin(g))/(1+meas_res*exp_z_anc))
        side=torch.clamp(side,min=-1,max=1)
        new_angles=2*torch.arcsin(side)
        angles.copy_(new_angles)

        return meas_res

    @staticmethod
    def backward(ctx,grad_output):
        preacts,=ctx.saved_tensors
        ones=torch.ones_like(preacts)
        ind=torch.logical_and(preacts>-ones,preacts<ones).float()
        return ind*grad_output, None, None, None, None

class WMeasure_Rot(nn.Module):
    def __init__(self,angles,last_res,a,g):
        super(WMeasure_Rot,self).__init__()
        self.a=torch.tensor(a)
        self.g=torch.tensor(g)
        self.angles=angles
        self.last_res=last_res
    def forward(self,input):
        return WMeasure_Rot_Fun.apply(input,self.angles,self.last_res,self.a,self.g)

class StepHTFunction(autograd.Function):
    @staticmethod
    def forward(ctx, input):
        ctx.save_for_backward(input)
        return ((input > 0).float()-0.5)*2
    @staticmethod
    def backward(ctx, grad_output):
        input,=ctx.saved_tensors
        ones=torch.ones(input.size())
        ind=torch.logical_and(input>-ones,input<ones).float()
        return ind*grad_output

class StepHT(nn.Module):
    def __init__(self):
        super(StepHT, self).__init__()
    def forward(self, x):
        x = StepHTFunction.apply(x)
        return x

class FCN(nn.Module):
    def __init__(self,width,depth,in_size,out_size):
        super(FCN,self).__init__()
        self.width=width
        self.angles=torch.tensor([])
        self.last_res=torch.tensor([])
        
        layers=[nn.Linear(in_size,width,bias=bias)]
        for _ in range(depth-2):
            layers.append(WMeasure_Rot(self.angles,self.last_res,a,g))
            layers.append(nn.Linear(width,width,bias=bias))
        layers.append(WMeasure_Rot(self.angles,self.last_res,a,np.pi/2))
        layers.append(nn.Linear(width,out_size,bias=bias))
        self.fcn=nn.Sequential(*layers)

    def prepare(self,x):
        temp_angles=torch.zeros((x.shape[0],self.width))
        self.angles.resize_(temp_angles.shape).copy_(temp_angles)
    
        temp_res=torch.ones((x.shape[0],self.width))
        self.last_res.resize_(temp_res.shape).copy_(torch.ones_like(temp_res))
    
    def forward(self,x):
        self.prepare(x)
        return self.fcn(x)

class Det_FCN(nn.Module):
  def __init__(self,width,depth,in_size,out_size):
    super(Det_FCN,self).__init__()
    layers=[nn.Linear(in_size,width,bias=bias),StepHT()]
    for _ in range(depth-2):
      layers.append(nn.Linear(width,width,bias=bias))
      layers.append(StepHT())
    layers.append(nn.Linear(width,out_size,bias=bias))
    # layers.append(act_fun(a))
    self.fcn=nn.Sequential(*layers)
    return

  def forward(self,x):
    return self.fcn(x)

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight,nonlinearity='linear')

train_set = datasets.MNIST('data/', train=True, download=True, 
                            transform=transforms.Compose([
                               transforms.ToTensor(),
                                torch.flatten
                             ]))
train_set=torch.utils.data.Subset(train_set,range(0,train_size))
train_loader = torch.utils.data.DataLoader(train_set,batch_size=batch_size_train, shuffle=True)

test_set = datasets.MNIST('data/', train=False, download=True,
                             transform=transforms.Compose([
                               transforms.ToTensor(),
                                 torch.flatten
                             ]))
test_set=torch.utils.data.Subset(test_set,range(0,test_size))
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size_test, shuffle=False)

def get_err(loader,model,num_shots):
    batch_err=[]
    for imgs, labels in loader:
        preds_coll=[torch.argmax(model(imgs),axis=1) for _ in range(num_shots)]
        preds=torch.tensor(scipy.stats.mode(preds_coll, axis=0, keepdims=False)[0])
        vec=(preds!=labels)
        batch_err.append(sum(vec)/len(preds))
    return np.mean(batch_err)

def get_err_y(ys,labels,num_shots):
    preds_coll=[torch.argmax(ys,axis=1) for _ in range(num_shots)]
    preds=torch.tensor(scipy.stats.mode(preds_coll, axis=0, keepdims=False)[0])
    vec=(preds!=labels)
    return sum(vec)/len(ys)

net=FCN(width,depth,input_size,output_size)
det_net=Det_FCN(width,depth,input_size,output_size)
net.apply(init_weights)
det_net.load_state_dict(net.state_dict())
loss=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=learning_rate,momentum=momentum)

first_params=[param.detach().tolist() for param in net.parameters()]
# param_list=[]
# params=[param.detach().tolist() for param in net.parameters()]
# param_list.append(params)
losses=[]
# rates=[scheduler.get_last_lr()]
err_rates=[float(get_err(test_loader,net,num_shots))]
train_err_rates=[float(get_err(train_loader,net,num_shots))]
det_err_rates=[float(get_err(test_loader,det_net,num_shots))]
det_net=FCN(width,depth,input_size,output_size)

first=True
step_count=0

for epoch in range(n_epochs):
    train_errs=[]
    for i, (imgs, labels) in enumerate(train_loader):
        imgs.requires_grad_(True)
        labels=labels.type(dtype=torch.long)
        optimizer.zero_grad()
        #forward pass
        Y_preds=net(imgs)
        if epoch%step==step-1:
            with torch.no_grad(): 
                train_errs.append(get_err_y(Y_preds,labels,1))

        loss_step=loss(Y_preds,labels)
        
        #comment out to not save first loss
        if first:
            losses.append(float(loss_step.detach().numpy()))
            first=False

        #backward pass
        loss_step.backward()
        optimizer.step()
        
    if epoch%step==step-1:
        losses.append(float(loss_step.detach().numpy()))
        # params=[param.detach().tolist() for param in net.parameters()]
        # param_list.append(params)

        with torch.no_grad():
            err_rates.append(float(get_err(test_loader,net,num_shots)))
            train_err_rates.append(float(np.mean(train_errs)))
        det_net.load_state_dict(net.state_dict())
        det_err_rates.append(float(get_err(test_loader,det_net,1)))

last_params=[param.detach().tolist() for param in net.parameters()]

with torch.no_grad():
    acc=1-get_err(test_loader,net,num_shots)

out_dict={
    "a": a,
    "g": g,
    "num_shots": num_shots,
    "train_size": train_size,
    "test_size": test_size,
    "learning_rate": learning_rate,
    "momentum": momentum,
    "batch_size": batch_size_train,
    "epochs": n_epochs,
    "input_size": input_size,
    "output_size": output_size,
    "depth": depth,
    "width": width,
    "test_accuracy": acc,
    "losses": losses,
    "err_rates": err_rates,
    "train_err_rates": train_err_rates,
    "det_err_rates": det_err_rates,
    "step": step,
    "first_params":first_params,
    "last_params":last_params
}

with open("data/test.json","w") as outfile:
    json.dump(out_dict,outfile)

print("acc:"+str(acc))


ModuleNotFoundError: No module named 'numpy'